In [1]:
using Pkg
Pkg.activate("/Users/ryansenne/Documents/Github/SSM/")
# Pkg.activate("/home/ryansenne/PycharmProjects/ssm_julia/")

  Activating project at `\Users\ryansenne\Documents\Github\SSM`


In [2]:
include("/Users/ryansenne/Documents/Github/SSM//src/SSM.jl")
# include("/home/ryansenne/PycharmProjects/ssm_julia/src/SSM.jl")
using Random
using .SSM
using Distributions

### Gaussian GLM Example

In [3]:
# Fit a Gaussian GLM
# Generate some synthetic data
X = rand(10000, 3);  # 100 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]
y = X_concat * β + rand(Normal(0, 1), 10000)

# define a GLM
glm = GaussianRegression(X, y)
fit!(glm, 1000)

4-element Vector{Float64}:
  1.963508062948609
  3.0472521214354646
 -4.032847152584493
  5.085586252239004

### Poisson GLM Example

In [4]:
# Fit a Poisson GLM
# Generate some synthetic data
X = rand(10000, 3);  # 100 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]

# Generate some synthetic data
y = Vector{Float64}(rand.(Poisson.(exp.(X_concat * β))))

# define a GLM
glm = PoissonRegression(X, y)
fit!(glm, 1000)


4-element Vector{Float64}:
  1.9972876910665238
  3.0018761978448465
 -4.001568346180793
  5.002617122785103

### Binomial GLM Example (i.e. logistic regression)

In [3]:
# sigmoid function
function sigmoid(x)
    return 1 / (1 + exp(-x))
end

# Fit a Binomial GLM
# Generate some synthetic data
X = rand(10000, 3); # 1000 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]

# Generate some synthetic data
y = Vector{Float64}(rand.(Binomial.(1, sigmoid.(X_concat * β))))

# define a GLM
glm = BinomialRegression(X, y)
fit!(glm, 1000)

4-element Vector{Float64}:
  1.8285295895751685
  3.0115236069793356
 -3.778705515037161
  5.156155128038023

# Autoregressive Model

In [3]:
# simulate AR(1) model
init_val = 0.0
phi = 0.9
sigma = 1.0
T = 1000
y = Vector{Float64}(undef, T)
y[1] = init_val
for t in 2:T
    y[t] = phi * y[t-1] + rand(Normal(0, sigma))
end

ar_1 = Autoregression(y, 1)
fit!(ar_1)

0.9538958584342558

In [15]:
# try out markov glm

# Generate synthetic data from two distinct regimes
X = rand(1000, 3); # 1000 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(1000), X)
β₁ = [-20, -30, -40, 50]
β₂ = [6, 7, 8, -9]

# Generate some synthetic data
y₁ = X_concat * β₁ + rand(Normal(0, 0.1), 1000)
y₂ = X_concat * β₂ + rand(Normal(0, 1), 1000)

# concatenate X1 and X2
x = vcat(X, X)

# concatenate y1 and y2
y = vcat(y₁, y₂)

# define a Gaussian HMMGLM
glm = SwitchingGaussianRegression(y, x, 2)

MarkovRegressionEM(glm)

Iteration: 1, Log-likelihood: -8924.586743524973
Iteration: 2, Log-likelihood: -8739.362326611385
Iteration: 3, Log-likelihood: -8590.645568136204
Iteration: 4, Log-likelihood: -8487.922486518475
Iteration: 5, Log-likelihood: -8425.134856905965
Iteration: 6, Log-likelihood: -8388.1560823782
Iteration: 7, Log-likelihood: -8366.508137598063
Iteration: 8, Log-likelihood: -8354.032129290421
Iteration: 9, Log-likelihood: -8347.05339618205
Iteration: 10, Log-likelihood: -8343.28945695505
Iteration: 11, Log-likelihood: -8341.336144244464
Iteration: 12, Log-likelihood: -8340.358723752062
Iteration: 13, Log-likelihood: -8339.884591585684
Iteration: 14, Log-likelihood: -8339.660157362405
Iteration: 15, Log-likelihood: -8339.55583983571
Iteration: 16, Log-likelihood: -8339.507983863976
Convergence reached at iteration 16
3.0470238249608047e12
2.736186558656916e12
Initial Variance: [1.5242740495051548e9, 1.3687776681625392e9]
NaN
NaN
Log-likelihood at iteration 1: -1.661341605283923e7
NaN
NaN
Log-

In [16]:
glm.B[2].regression_model.β

4-element Vector{Float64}:
 -19.8170073700559
 -28.185869452125555
 -37.88979160361808
  47.35209106013571

In [4]:
glm.σ²

2-element Vector{Float64}:
 NaN
 NaN

In [5]:
glm.B[2].regression_model.β

4-element Vector{Float64}:
 -20.058395420154884
 -28.413295346218813
 -38.62800924981708
  48.29198573140382

In [22]:
function inverse_tridiagonal(A)
    N = size(A, 1)
    
    # Initialize vectors and matrices
    D = zeros(N)
    E = zeros(N)
    Z = zeros(N, N)
    W = zeros(N, N)
    λ = zeros(N, N)

    # Boundary conditions
    D[1] = 0
    E[N] = 0
    Z[:, 1] = 0
    W[:, N] = 0

    for i = 2:N
        # Forward sweep for D and Z
        D[i] = (A[i, i] - A[i, i-1] * D[i-1])^-1
        for j = 1:i-1
            Z[i, j] = -D[i] * (A[i, i-1] * Z[i-1, j])
        end

        # Reverse sweep for E and W
        E[N-i+1] = (A[N-i+1, N-i+1] - A[N-i+1, N-i+2] * E[N-i+2])^-1
        for j = N:-1:N-i+2
            W[N-i+1, j] = -E[N-i+1] * (A[N-i+1, N-i+2] * W[N-i+2, j])
        end
    end

    # Compute the diagonal of the inverse
    for i = 1:N
        λ[i, i] = (1 - D[i] * E[i+1])^-1 * (A[i, i] - A[i, i-1] * D[i-1])^-1
        if i < N
            λ[i+1, i] = E[i+1] * λ[i, i]
            λ[i, i+1] = D[i] * λ[i+1, i+1] + Z[i, i+1]
        end
    end

    return λ
end

function inverse_block_tridiagonal(A_blocks, B_blocks, C_blocks)
    N = length(A_blocks)
    
    # Initialize block vectors and matrices
    D_blocks = [zeros(size(A_blocks[1])) for _ in 1:N]
    E_blocks = [zeros(size(A_blocks[1])) for _ in 1:N]
    λ_blocks = [zeros(size(A_blocks[1])) for _ in 1:N]
    
    # Forward sweep
    D_blocks[1] = inv(A_blocks[1])
    for i = 2:N
        D_blocks[i] = inv(A_blocks[i] - B_blocks[i-1] * D_blocks[i-1] * C_blocks[i-1])
    end

    # Reverse sweep
    E_blocks[N] = inv(A_blocks[N])
    for i = N-1:-1:1
        E_blocks[i] = inv(A_blocks[i] - C_blocks[i] * E_blocks[i+1] * B_blocks[i])
    end

    # Compute the diagonal blocks of the inverse
    λ_blocks[1] = inv(I - D_blocks[1] * E_blocks[2]) * D_blocks[1]
    for i = 2:N-1
        λ_blocks[i] = inv(I - D_blocks[i] * E_blocks[i+1]) * D_blocks[i]
    end
    λ_blocks[N] = inv(I - D_blocks[N] * E_blocks[N]) * D_blocks[N]
    
    # Create the inverse matrix from the block diagonals
    # inv_matrix = BlockDiagonal(λ_blocks)

    return λ_blocks
end


a = [[1 2; 0 0], [3 4; 5 0], [0 6; 7 8], [0 0; 9 10]]

b = [[1 2; 0 0], [3 4; 5 0], [0 6; 7 8], [0 0; 9 10]]

c = [[1 2; 0 0], [3 4; 5 0], [0 6; 7 8], [0 0; 9 10]]

inverse_block_tridiagonal(a, b, c)


LinearAlgebra.LAPACKException: LinearAlgebra.LAPACKException(2)